## Loading libraries

In [1]:
import pandas as pd
import numpy as np

## Loading data

In [2]:
demand_val_actual = pd.read_csv("./Processed Data/Demand_Val.csv", header=None)
solar_val_actual = pd.read_csv("./Processed Data/Solar_Val.csv", header=None)
price_val_actual = pd.read_csv("./Processed Data/Price_Val.csv", header=None)
bid_price = pd.read_csv("./Output/BidPrice.csv", header=None)
bid_quantity = pd.read_csv("./Output/BidQuantity.csv", header=None)

In [3]:
bill = pd.DataFrame(np.random.rand(50,24))

## Calculating the daily bill

1. If bid price is greater than the market price, we buy the corresponding quantity from the market and charge or discharge the battery according to the solar output and the total demand of the block. If demand is greater than combined bid quantity, solar and battery, we buy electricity from discom.

2. If bid price is less than the market price, we loose the bid and have to take electricity from battery and solar output. If that is not sufficient, we have to buy it from discom.

In [4]:
battery = np.float64(0.0)
for i in range(50):
    for j in range(24):
        block_bill = np.float64(0.0)
        if(bid_price.loc[i,j] > price_val_actual.loc[i,j]):
            block_bill=block_bill + bid_price.loc[i,j]*bid_quantity.loc[i,j]
            if(bid_quantity.loc[i,j] > demand_val_actual.loc[i,j] or bid_quantity.loc[i,j] + solar_val_actual.loc[i,j] > demand_val_actual.loc[i,j]):
                if(battery<np.float64(25.0)):
                    if(bid_quantity.loc[i,j] - demand_val_actual.loc[i,j] + solar_val_actual.loc[i,j] <= np.float64(5.0)):
                        battery = battery + bid_quantity.loc[i,j] - demand_val_actual.loc[i,j] + solar_val_actual.loc[i,j]
                    else:
                        battery=battery + np.float64(5.0)
                    if(battery > np.float64(25.0)):
                        battery = np.float64(25.0)
                else:
                    battery = np.float64(25.0)
            else:
                if(battery <= np.float64(5.0)):
                    if(demand_val_actual.loc[i,j] > bid_quantity.loc[i,j] + solar_val_actual.loc[i,j] + np.float64(0.8) * battery):
                        block_bill = block_bill + (np.float64(7.0) * (demand_val_actual.loc[i,j] - bid_quantity.loc[i,j] - solar_val_actual.loc[i,j] - np.float64(0.8)*battery))
                        battery = np.float64(0.0)
                    else:
                        battery = battery - (np.float64(1.25) * (demand_val_actual.loc[i,j] - bid_quantity.loc[i,j] - solar_val_actual.loc[i,j]))
                else:
                    if(demand_val_actual.loc[i,j] > bid_quantity.loc[i,j] + solar_val_actual.loc[i,j] + np.float64(4.0)):
                        block_bill = block_bill + (np.float64(7.0) * (demand_val_actual.loc[i,j] - bid_quantity.loc[i,j] - solar_val_actual.loc[i,j] - np.float64(4.0)))
                        battery = battery - np.float64(5.0)
                    else:
                        battery = battery - (np.float64(1.25) * (demand_val_actual.loc[i,j] - bid_quantity.loc[i,j] - solar_val_actual.loc[i,j]))
        else:
            if(solar_val_actual.loc[i,j] > demand_val_actual.loc[i,j]):
                if(battery < np.float64(25.0)):
                    if(solar_val_actual.loc[i,j] - demand_val_actual.loc[i,j] <= np.float64(5.0)):
                        battery = battery + solar_val_actual.loc[i,j] - demand_val_actual.loc[i,j]
                    else:
                        battery = battery+np.float64(5.0)
                    if(battery > np.float64(25.0)):
                        battery = np.float64(25.0)
                else:
                    battery = np.float64(25.0)
            else:
                if(battery <= np.float64(5.0)):
                    if(demand_val_actual.loc[i,j] > solar_val_actual.loc[i,j] + np.float64(0.8) * battery):
                        block_bill = block_bill + (np.float64(7.0) * (demand_val_actual.loc[i,j] - solar_val_actual.loc[i,j] - np.float64(0.8) * battery))
                        battery = np.float64(0.0)
                    else:
                        battery = battery - (np.float64(1.25) * (demand_val_actual.loc[i,j] - solar_val_actual.loc[i,j]))
                else:
                    if(demand_val_actual.loc[i,j] > solar_val_actual.loc[i,j] + np.float64(4.0)):
                        block_bill = block_bill + (np.float64(7.0) * (demand_val_actual.loc[i,j] - solar_val_actual.loc[i,j] - np.float64(4.0)))
                        battery = battery - np.float64(5.0)
                    else:
                        battery = battery - (np.float64(1.25) * (demand_val_actual.loc[i,j] - solar_val_actual.loc[i,j]))
        bill.loc[i,j] = block_bill

In [5]:
bill

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,205.6600,170.2094,431.7427,232.9343,146.4808,150.8430,541.0074,1302.8955,940.1924,511.3262,...,965.1810,644.6254,682.8524,138.9782,494.9733,1393.7575,1184.4790,908.8380,305.4320,247.7830
1,168.4767,218.3970,143.6561,309.5924,293.8436,176.0220,419.1040,1593.7967,1183.7070,933.7745,...,389.3520,364.7030,838.5969,749.4932,413.7110,1336.4388,1558.1220,1496.6639,1294.7500,955.6890
2,612.6796,605.2138,165.0288,153.3396,151.0840,152.2097,275.2293,1006.9912,854.3304,1286.1660,...,775.4128,222.8520,216.4510,216.6451,1401.9040,1557.1271,1569.5680,1231.2700,986.0636,388.8380
3,201.0260,355.1020,323.5156,184.2483,193.8336,283.9416,406.1352,710.0432,1017.9140,1566.7372,...,1891.8310,1795.6676,945.4000,1376.4717,1187.6524,1175.7679,1512.2676,1181.1000,962.6390,1050.6392
4,795.1088,274.7022,161.0466,155.2854,179.2574,209.7284,955.4212,1852.7468,1214.3212,1206.3821,...,519.9060,411.2446,210.4668,265.4320,444.3687,615.2476,1686.7528,1034.1800,414.0178,227.2508
5,236.0511,206.9567,270.0552,202.7670,299.0368,336.7805,398.1818,1359.7061,1040.5890,1012.8582,...,194.5251,65.8112,195.1222,218.9020,602.3180,1229.3050,617.1972,1016.5220,1384.2360,237.3592
6,292.4649,190.8238,180.0483,189.4348,187.9280,177.2860,790.9890,1063.1136,1788.8528,1166.1786,...,762.8160,818.2273,639.0716,602.7700,920.8744,903.0244,1639.8920,914.8804,662.7915,307.7195
7,345.4050,205.1784,476.4942,169.8416,147.3530,269.2601,524.4250,1464.6780,987.9563,1114.8196,...,593.0620,572.3158,845.2580,1041.2292,1020.9288,1051.5788,986.5640,1199.9064,774.3008,476.9540
8,329.0029,183.3370,181.2972,327.8800,303.6990,216.2272,497.5688,1474.5850,725.9973,713.9192,...,617.4625,612.5354,831.2310,900.6456,1058.7967,1537.3064,1387.8852,1487.4850,1519.0088,773.6456
9,442.6050,175.9665,208.1569,200.9280,194.8010,172.0433,190.5900,327.7001,456.7800,681.7529,...,977.2189,585.4531,1009.9504,1477.3374,1480.8800,1352.8952,1483.5876,932.3284,483.8596,298.0541


In [6]:
daily_bill = pd.DataFrame(bill.sum(axis=1), columns=['Daily Bill'])
daily_bill

,Daily Bill
0,14713.5995
1,16836.2565
2,17304.1853
3,23301.4592
4,15621.3558
5,12992.4345
6,17842.6478
7,17530.5070
8,18967.2301
9,17503.7667


In [7]:
print('The total bill is {:.2f}.'.format(daily_bill.sum()[0]))

The total bill is 816138.75.
